## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/Sample-2.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

ID,Name,Marks
1,Sagar,50
2,Shivam,40
3,Muni,20
1,Sagar,80
2,Shivam,10
3,Muni,90


In [0]:
df1 = df.groupBy('ID','Name').sum('Marks')
df1 = df1.withColumnRenamed('sum(Marks)','Marks_Sum')
df1.show()

+---+------+---------+
| ID|  Name|Marks_Sum|
+---+------+---------+
|  2|Shivam|       50|
|  3|  Muni|      110|
|  1| Sagar|      130|
+---+------+---------+



In [0]:
df1.write.csv("/FileStore/tables_output/Sample-3.csv")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2464622716046954> in <module>
----> 1 df1.write.csv("/FileStore/tables_output/Sample-3.csv")

/databricks/spark/python/pyspark/sql/readwriter.py in csv(self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep)
    953                        charToEscapeQuoteEscaping=charToEscapeQuoteEscaping,
    954                        encoding=encoding, emptyValue=emptyValue, lineSep=lineSep)
--> 955         self._jwrite.csv(path)
    956 
    957     def orc(self, path, mode=None, partitionBy=None, compression=None):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.s

In [0]:
#overwriting the existing file
df1.write.mode("overwrite").csv("/FileStore/tables_output/Sample-3.csv")

In [0]:
df1.write.mode("append").csv("/FileStore/tables_output/Sample-3.csv")

In [0]:
df_output = spark.read.csv('/FileStore/tables_output/Sample-3.csv/part-00000-tid-2646713785346887290-1a3b75f6-216d-4c92-a88c-729541ca59eb-6-1-c000.csv')
df_output.show()

+---+------+---+
|_c0|   _c1|_c2|
+---+------+---+
|  2|Shivam| 40|
|  3|  Muni| 90|
|  1| Sagar| 80|
+---+------+---+



In [0]:
df_output = spark.read.csv('/FileStore/tables_output/Sample-3.csv/part-00000-tid-202569500061172705-e4dd9cc3-7e06-4804-95a8-2a4f99552963-14-1-c000.csv')
df_output.show()

+---+------+---+
|_c0|   _c1|_c2|
+---+------+---+
|  2|Shivam| 50|
|  3|  Muni|110|
|  1| Sagar|130|
+---+------+---+



In [0]:
df_output = spark.read.csv('/FileStore/tables_output/Sample-3.csv/part-00000-tid-6205794230178144312-a354cc31-5f3c-4d29-a121-e5d7920807b2-12-1-c000.csv')
df_output.show()

+---+------+---+
|_c0|   _c1|_c2|
+---+------+---+
|  2|Shivam| 50|
|  3|  Muni|110|
|  1| Sagar|130|
+---+------+---+



In [0]:
df_output = spark.read.csv('/FileStore/tables_output/Sample-3.csv/part-00000-tid-3039071969243927029-4f820449-5da0-4209-ae9e-a9102168aeb1-20-1-c000.csv')
df_output.show()

+---+------+---+
|_c0|   _c1|_c2|
+---+------+---+
|  2|Shivam| 50|
|  3|  Muni|110|
|  1| Sagar|130|
+---+------+---+



In [0]:
 # Create a view or table

temp_table_name = "Sample-2_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `Sample-2_csv`

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "Sample-2_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)